# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import sqlite3
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * from disaster_response', con = engine)

#seperating the dataframe into labels and features
X = df['message']
Y = df[['related', 'request', 'offer', 'aid_related', 'medical_help',
        'medical_products', 'search_and_rescue', 'security', 'military',
        'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
        'missing_people', 'refugees', 'death', 'other_aid',
        'infrastructure_related', 'transport', 'buildings', 'electricity',
        'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
        'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
        'other_weather', 'direct_report']]
category_names = Y.columns

In [3]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
X.describe()

count                                                 26140
unique                                                26140
top       The recue work is still being done using manua...
freq                                                      1
Name: message, dtype: object

In [5]:
Y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.0,...,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000
mean,0.773374,0.170352,0.004476,0.413963,0.079457,0.050077,0.027659,0.018018,0.032785,0.0,...,0.011821,0.043879,0.278194,0.081943,0.093229,0.010788,0.093688,0.020161,0.052525,0.193344
std,0.435409,0.375949,0.066754,0.492551,0.270456,0.218107,0.163996,0.133020,0.178077,0.0,...,0.108082,0.204830,0.448118,0.274284,0.290758,0.103306,0.291400,0.140552,0.223087,0.394928
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
Y.shape

(26140, 36)

In [7]:
X.shape

(26140,)

In [8]:
Y.loc[Y['related']==2]


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
458,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
878,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
892,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
920,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
926,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
Y.related=Y.related.replace(2,1)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
Y.related.value_counts()

1    20029
0     6111
Name: related, dtype: int64

In [11]:
Y.describe()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.0,...,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000,26140.000000
mean,0.766220,0.170352,0.004476,0.413963,0.079457,0.050077,0.027659,0.018018,0.032785,0.0,...,0.011821,0.043879,0.278194,0.081943,0.093229,0.010788,0.093688,0.020161,0.052525,0.193344
std,0.423242,0.375949,0.066754,0.492551,0.270456,0.218107,0.163996,0.133020,0.178077,0.0,...,0.108082,0.204830,0.448118,0.274284,0.290758,0.103306,0.291400,0.140552,0.223087,0.394928
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 2. Write a tokenization function to process your text data

In [12]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [13]:
def tokenize(message):
    detected_urls = re.findall(url_regex, message)
    
    for url in detected_urls:
        message=message.replace(url, "urlplaceholder")
    tokens = word_tokenize(message)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [14]:
for mes in X[:5]:
    tokens = tokenize(mes)
    print(mes)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0)))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

In [17]:
Y_pred.shape

(6535, 36)

In [18]:
Y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [19]:
Y_preds=pd.DataFrame(Y_pred)
Y_preds

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
Y_test


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
11131,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1304,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
5486,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1843,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14186,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
18274,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13175,1,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
20514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
print(classification_report(np.ravel(Y_test), np.ravel(Y_pred)))

             precision    recall  f1-score   support

          0       0.95      0.99      0.97    214425
          1       0.81      0.44      0.57     20835

avg / total       0.94      0.94      0.93    235260



In [22]:
for i, col in enumerate(Y.columns):
    print("Colummn : {}".format(col))
    Y_true = Y_test.values[:, i]
    print(Y_true)

Colummn : related
[1 0 1 ..., 1 1 1]
Colummn : request
[0 0 0 ..., 0 0 1]
Colummn : offer
[0 0 0 ..., 0 0 0]
Colummn : aid_related
[1 0 1 ..., 1 1 1]
Colummn : medical_help
[0 0 0 ..., 1 0 0]
Colummn : medical_products
[0 0 0 ..., 0 0 0]
Colummn : search_and_rescue
[0 0 0 ..., 0 0 0]
Colummn : security
[0 0 0 ..., 0 0 0]
Colummn : military
[0 0 0 ..., 0 0 0]
Colummn : child_alone
[0 0 0 ..., 0 0 0]
Colummn : water
[0 0 0 ..., 0 0 0]
Colummn : food
[1 0 0 ..., 0 0 1]
Colummn : shelter
[0 0 0 ..., 0 0 0]
Colummn : clothing
[0 0 0 ..., 0 0 0]
Colummn : money
[0 0 0 ..., 0 0 0]
Colummn : missing_people
[0 0 0 ..., 0 0 0]
Colummn : refugees
[0 0 0 ..., 0 0 0]
Colummn : death
[0 0 0 ..., 0 0 0]
Colummn : other_aid
[1 0 1 ..., 0 1 0]
Colummn : infrastructure_related
[0 0 0 ..., 0 0 0]
Colummn : transport
[0 0 0 ..., 0 0 0]
Colummn : buildings
[0 0 0 ..., 0 0 0]
Colummn : electricity
[0 0 0 ..., 0 0 0]
Colummn : tools
[0 0 0 ..., 0 0 0]
Colummn : hospitals
[0 0 0 ..., 0 0 0]
Colummn : shops
[0

In [23]:
for i, col in enumerate(Y.columns):
    print("Colummn : {}".format(col))
    Y_predss = Y_preds.values[:, i]
    print(Y_predss)

Colummn : related
[1 1 1 ..., 1 1 1]
Colummn : request
[0 0 0 ..., 0 0 1]
Colummn : offer
[0 0 0 ..., 0 0 0]
Colummn : aid_related
[0 0 0 ..., 1 0 0]
Colummn : medical_help
[0 0 0 ..., 0 0 0]
Colummn : medical_products
[0 0 0 ..., 0 0 0]
Colummn : search_and_rescue
[0 0 0 ..., 0 0 0]
Colummn : security
[0 0 0 ..., 0 0 0]
Colummn : military
[0 0 0 ..., 0 0 0]
Colummn : child_alone
[0 0 0 ..., 0 0 0]
Colummn : water
[0 0 0 ..., 0 0 0]
Colummn : food
[0 0 0 ..., 0 0 0]
Colummn : shelter
[0 0 0 ..., 0 0 0]
Colummn : clothing
[0 0 0 ..., 0 0 0]
Colummn : money
[0 0 0 ..., 0 0 0]
Colummn : missing_people
[0 0 0 ..., 0 0 0]
Colummn : refugees
[0 0 0 ..., 0 0 0]
Colummn : death
[0 0 0 ..., 0 0 0]
Colummn : other_aid
[0 0 0 ..., 0 0 0]
Colummn : infrastructure_related
[0 0 0 ..., 0 0 0]
Colummn : transport
[0 0 0 ..., 0 0 0]
Colummn : buildings
[0 0 0 ..., 0 0 0]
Colummn : electricity
[0 0 0 ..., 0 0 0]
Colummn : tools
[0 0 0 ..., 0 0 0]
Colummn : hospitals
[0 0 0 ..., 0 0 0]
Colummn : shops
[0

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
#classification report for the entire dataset
print(classification_report(np.ravel(Y_test), np.ravel(Y_pred)))

             precision    recall  f1-score   support

          0       0.95      0.99      0.97    214425
          1       0.81      0.44      0.57     20835

avg / total       0.94      0.94      0.93    235260



In [26]:
#classification report got by looping through all the columns
for i, col in enumerate(Y.columns):
    print("Colummn : {}".format(col))
    Y_true = Y_test.values[:, i]
    Y_predss = Y_preds.values[:, i]
    target_names = ['is_{}'.format(col), 'not_{}'.format(col)]
    print(classification_report(Y_true, Y_predss, target_names=target_names))
    

Colummn : related
             precision    recall  f1-score   support

 is_related       0.60      0.36      0.45      1513
not_related       0.83      0.93      0.88      5022

avg / total       0.78      0.80      0.78      6535

Colummn : request
             precision    recall  f1-score   support

 is_request       0.89      0.98      0.93      5433
not_request       0.80      0.38      0.52      1102

avg / total       0.87      0.88      0.86      6535

Colummn : offer
             precision    recall  f1-score   support

   is_offer       1.00      1.00      1.00      6505
  not_offer       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6535

Colummn : aid_related
                 precision    recall  f1-score   support

 is_aid_related       0.72      0.87      0.79      3841
not_aid_related       0.74      0.52      0.61      2694

    avg / total       0.73      0.73      0.72      6535

Colummn : medical_help
                  precision

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
def build_model():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
        #"vect_ngram_range":((1, 1), (1, 2)),
        #"vect_max_df":(0.5, 0.75),
        #"vect_max_features":(None, 5000, 10000),
        "tfidf__use_idf":(True, False),
        "clf__estimator__n_estimators": [10, 20, 50],
        #"clf_min_samples_split": [2, 3, 4]
    }   
    cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs=4, verbose=2)
    
    return cv

In [28]:
model=build_model()
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, total= 3.4min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, total= 3.5min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, total= 3.5min
[CV] clf__estimator__n_estimators=20, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, total= 3.5min
[CV] clf__estimator__n_estimators=20, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, total= 3.3min


[Parallel(n_jobs=4)]: Done  18 out of  18 | elapsed: 37.2min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
#Classification report for the entire dataset
print(classification_report(np.ravel(Y_test), np.ravel(Y_pred)))

             precision    recall  f1-score   support

          0       0.95      0.99      0.97    214425
          1       0.82      0.48      0.60     20835

avg / total       0.94      0.94      0.94    235260



In [30]:
Y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [31]:
Y_preds=pd.DataFrame(Y_pred)
Y_preds

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#classification report got by looping through each column after applying grid search
for i, col in enumerate(Y.columns):
    print("Colummn : {}".format(col))
    Y_true = Y_test.values[:, i]
    Y_predss = Y_preds.values[:, i]
    target_names = ['is_{}'.format(col), 'not_{}'.format(col)]
    print(classification_report(Y_true, Y_predss, target_names=target_names))
    

Colummn : related
             precision    recall  f1-score   support

 is_related       0.73      0.26      0.38      1513
not_related       0.81      0.97      0.89      5022

avg / total       0.79      0.81      0.77      6535

Colummn : request
             precision    recall  f1-score   support

 is_request       0.90      0.99      0.94      5433
not_request       0.87      0.43      0.58      1102

avg / total       0.89      0.89      0.88      6535

Colummn : offer
             precision    recall  f1-score   support

   is_offer       1.00      1.00      1.00      6505
  not_offer       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6535

Colummn : aid_related
                 precision    recall  f1-score   support

 is_aid_related       0.75      0.88      0.81      3841
not_aid_related       0.77      0.59      0.67      2694

    avg / total       0.76      0.76      0.75      6535

Colummn : medical_help
                  precision

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
#trying other machine learning algorithms
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=0)))
])


In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

In [35]:
#overall classification report for other machine learning algorithms
print(classification_report(np.ravel(Y_test), np.ravel(Y_pred)))

             precision    recall  f1-score   support

          0       0.96      0.98      0.97    215036
          1       0.77      0.58      0.66     20224

avg / total       0.95      0.95      0.95    235260



In [36]:
#adding other features besides tfidf
import nltk
nltk.download('averaged_perceptron_tagger')
  
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, message):
        sentence_list = nltk.sent_tokenize(message)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [37]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
])


In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

In [39]:
#overall classification report
print(classification_report(np.ravel(Y_test), np.ravel(Y_pred)))

             precision    recall  f1-score   support

          0       0.94      0.99      0.97    214839
          1       0.83      0.39      0.53     20421

avg / total       0.93      0.94      0.93    235260



### 9. Export your model as a pickle file

In [40]:
import pickle

def export_model(model):
    filename = 'classifier.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    #load the model from disk
    #loaded_model = pickle.load(open(filename, 'rb'))
    #Y_pred = loaded_model.predict(X_test)
    #result = classification_report(Y_test, Y_pred, target_names=category_names)
    #print(result)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
script files done in IDE workspace